## Difine functions

Define some parameters and generators

In [1]:
sigma4 = Join[{IdentityMatrix[2]}, Table[PauliMatrix[i], {i, 1, 3}]];
sigmabar4 = Join[{IdentityMatrix[2]}, -Table[PauliMatrix[i], {i, 1, 3}]];
su11sigma = {PauliMatrix[3], I PauliMatrix[1], I PauliMatrix[2]};

In [4]:
Jvec = {{{0, 1, 0, 0}, {1, 0, 0, 0}, {0, 0, 0, 0}, {0, 0, 0, 0}}, {{0,
      0, 1, 0}, {0, 0, 0, 0}, {1, 0, 0, 0}, {0, 0, 0, 0}}, {{0, 0, 0, 
     1}, {0, 0, 0, 0}, {0, 0, 0, 0}, {1, 0, 0, 
     0}}, -{{0, 0, 0, 0}, {0, 0, 0, 0}, {0, 0, 0, -1}, {0, 0, 1, 
      0}}, -{{0, 0, 0, 0}, {0, 0, 0, 1}, {0, 0, 0, 0}, {0, -1, 0, 
      0}}, {{0, 0, 0, 0}, {0, 0, 1, 0}, {0, -1, 0, 0}, {0, 0, 0, 0}}};

In [5]:
jjvec = Join[1/2 Table[PauliMatrix[i], {i, 1, 3}], I/2 Table[PauliMatrix[i], {i, 1, 3}]];

In [6]:
η = DiagonalMatrix[{-1, 1, 1, 1}];

In [7]:
edgeor = Subsets[#, {2}] & /@ Subsets[{1, 2, 3, 4, 5}, {4}];
faceor = Subsets[#, {3}] & /@ Subsets[{1, 2, 3, 4, 5}, {4}];

In [9]:
etof = {{1, 2}, {1, 3}, {2, 3}, {4, 5}};
tetorder = Table[j, {j, 5, 1, -1}];
edgefromfaceor = Table[Subsets[#, {2}] & /@ faceor[[i]], {i, 1, 5}];
edgefromfaceornum = Table[Position[edgeor[[i]], #] & /@ edgefromfaceor[[i, j]]//Flatten, {i, 1, 5}, {j, 1, 4}];

Define functions

In [13]:
normso13[aa_, bb_] := aa . η . bb

In [14]:
gettetnormalvec[edgetet_] := Block[{a, b, c, d},
  FullSimplify[{a, b, c, d}/
    Sqrt[Abs[normso13[{a, b, c, d}, {a, b, c, d}]]] /. 
     Solve[
      Thread[({a, b, c, d} . DiagonalMatrix[{-1, 1, 1, 1}] . # & /@ 
          edgetet[[1 ;; 3]]) == 0], {a, b, c, d}] // Flatten, 
   Join[{a > 0, b > 0, c > 0, d > 0}, 
    Thread[{a, b, c, d} ϵ Reals]]]
  ]

In [15]:
gettetoutnormalvec1[edgetetall_, bdypoint_] := 
 Block[{unsignnormalvec$111 = gettetnormalvec /@ edgetetall},
  Table[If[
    unsignnormalvec$111[[i]] . η . unsignnormalvec$111[[i]] > 0, 
    unsignnormalvec$111[[i]] Sign[
      unsignnormalvec$111[[
         i]] . (bdypoint[[tetorder[[i]]]] - 
          bdypoint[[edgeor[[i, 1, 1]]]]) // N], 
    Sign[unsignnormalvec$111[[i, 1]] // N] unsignnormalvec$111[[
      i]]], {i, Length[unsignnormalvec$111]}]
  ]

In [16]:
vectospinor[aa_] := {{aa[[1]] + aa[[4]], aa[[2]] - I aa[[3]]}, {aa[[2]] + I aa[[3]], aa[[1]] - aa[[4]]}}

In [17]:
getreim[ff_, assume_] := Simplify[({Re[#], Im[#]} & @ ff // ExpandAll // FunctionExpand), Assumptions -> assume]

In [18]:
getsl2c[n1_, n2_] := 
 Block[{sn2 = vectospinor[n2], sn1 = vectospinor[n1], ggi, soleqggisn,
    ggire, a1, a2, b1, b2, c1, c2, d1, d2, eqggisn, symbolsoleqggisn, 
   symbolinstancesoleqggisn},
  ggi = FullSimplify[{{a1 + I a2 , b1 + I b2}, {c1 + I c2, d1 + I d2}} . sn1 . 
  ConjugateTranspose[{{a1 + I a2 , b1 + I b2}, {c1 + I c2, d1 + I d2}}], {a1, a2, b1, b2, c1, c2, d1, d2} ∈ Reals]; 
  eqggisn = 
   DeleteCases[
    Thread[(getreim[
          ggi, {a1, a2, b1, b2, c1, c2, d1, d2} ∈ Reals] - 
         getreim[
          sn2, {a1, a2, b1, b2, c1, c2, d1, d2} ∈ Reals] // 
        Flatten) == 0], True];
  soleqggisn = 
   Solve[
     Join[
      eqggisn, {Det[{{a1 + I a2 , b1 + I b2}, {c1 + I c2, 
           d1 + I d2}}] == 1}], {a1, a2, b1, b2, c1, c2, d1, d2}][[1]];
  symbolsoleqggisn = 
   DeleteDuplicates@
    Cases[{a1, a2, b1, b2, c1, c2, d1, d2} /. soleqggisn, _Symbol, -1];
  symbolinstancesoleqggisn = 
   FindInstance[
     Thread[
      DeleteDuplicates@Cases[soleqggisn, Sqrt[xx_] -> xx, -1] > 0], 
     symbolsoleqggisn, Reals][[1]];
  ggire = {{a1 + I a2 , b1 + I b2}, {c1 + I c2, d1 + I d2}} /. 
     soleqggisn /. symbolinstancesoleqggisn]

In [19]:
getsl2c[nn_] := 
 Block[{lsign$111 = Round[normso13[#, #]] & @ nn}, 
  If[lsign$111 === (normso13[#, #] & @ ({1, 0, 0, 0})), 
   getsl2c[{1, 0, 0, 0}, nn], 
   If[lsign$111 === (normso13[#, #] & @ ({0, 0, 0, 1})), 
    getsl2c[{0, 0, 0, 1}, nn]]]]

In [20]:
getso13[sl2cg_] := 
 Table[1/2 Tr[
    sl2cg . sigmabar4[[i]] . ConjugateTranspose[sl2cg] . 
     sigmabar4[[j]]], {i, 1, 4}, {j, 1, 4}]

In [21]:
get3dtet[tetlenthvec_, tetsol13_] := 
 Block[{tet3dvec$111, zeropos}, 
  tet3dvec$111 = FullSimplify[tetsol13 . # & /@ tetlenthvec]; 
  zeropos = Position[Chop@Total[Abs[tet3dvec$111 // N]], 0][[1]]; 
  If[Length[zeropos] === 1, {Drop[#, zeropos] & /@ tet3dvec$111, 
    zeropos}, tet3dvec$111]]

In [22]:
getfacevec[{tetedgevec_, normali_}] := 
 Block[{meta$111 = 
    Drop[η, {normali} // Flatten, {normali} // 
      Flatten]}, {1/2 Cross[meta$111 . tetedgevec[[#[[1]]]], 
       meta$111 . tetedgevec[[#[[2]]]]] & /@ etof, meta$111}]

In [23]:
getoutpfacevec[{tetedgevec_, normali_}] := 
 Block[{rrr$111 = getfacevec[{tetedgevec, normali}], nvec$111, 
   meta$111}, nvec$111 = rrr$111[[1]]; 
  meta$111 = 
   rrr$111[[
    2]]; {(Sign /@ (Table[{tetedgevec[[3]], tetedgevec[[2]], 
            tetedgevec[[1]], -tetedgevec[[1]]}[[i]] . meta$111 . 
          nvec$111[[i]], {i, Length[nvec$111]}] // N)) nvec$111, 
   meta$111}]

In [24]:
getfacenormal[{facevec_, meta_}] := #/Sqrt[Abs[(# . meta . # )]] & /@ facevec

In [25]:
getarea[{facevec_, meta_}] := Sqrt[Abs[(# . meta . # )]] & /@ facevec

In [26]:
threetofour[n_, sgndet1_] := If[sgndet1 > 0, Join[{0}, n], Join[n, {0}]]

In [27]:
getbivec[n1_, n2_] := 1/2 Table[Sum[LeviCivitaTensor[4][[i, j, k, l]] (η . n1)[[k]] (η . n2)[[l]], {k, 1, 4}, {l, 1, 4}], {i, 1, 4}, {j, 1, 4}] . η

In [28]:
bivec1tohalf[bivec_] := 
 Total[ 1/2 Join[
    Table[
     Tr[bivec . Jvec[[i]]], {i, 1, 3}], -Table[
      Tr[bivec . Jvec[[i]]], {i, 4, 6}]]*jjvec]

In [29]:
Off[Solve::svars]
Off[Solve::ratnz]

eqssu2 = 
  DeleteCases[
   DeleteDuplicates[
    Thread[(FullSimplify[{Re[#], Im[#]} & /@ 
           ExpandAll[
            Join[
               Flatten[(({{a$eqt, b$eqt}, {-Conjugate[b$eqt], 
                    Conjugate[a$eqt]}} . PauliMatrix[3] . 
                    Inverse[{{a$eqt, b$eqt}, {-Conjugate[b$eqt], 
                    Conjugate[a$eqt]}}] - 
                    Total[{ee$eqt, ff$eqt, gg$eqt}*
                    Table[PauliMatrix[i], {i, 1, 3}]] /. 
                   a$eqt Conjugate[a$eqt] + b$eqt Conjugate[b$eqt] -> 
                    1))], {a$eqt Conjugate[a$eqt] + 
                 b$eqt Conjugate[b$eqt] - 1}] /. 
              a$eqt -> aa1$eqt + I aa2$eqt /. 
             b$eqt -> bb1$eqt + I bb2$eqt], {aa1$eqt, aa2$eqt, 
            bb1$eqt, bb2$eqt, ee$eqt, ff$eqt, gg$eqt} ∈ 
           Reals] // Flatten) == 0] // FullSimplify], True];

soleqssu2$eqs = (Solve[
        eqssu2[[1 ;; 3]], {aa1$eqt, aa2$eqt, bb1$eqt, bb2$eqt}] /. 
       ee$eqt^2 + ff$eqt^2 + gg$eqt^2 -> 1 // FullSimplify) //. 
    ff$eqt^2 -> 1 - ee$eqt^2 - gg$eqt^2 // FullSimplify;

soleqssu2 = 
  soleqssu2$eqs[[
   Position[
     Total[
      Transpose@
       Boole[(eqssu2 /. soleqssu2$eqs //. 
             ff$eqt^2 -> 1 - ee$eqt^2 - gg$eqt^2 // FullSimplify) //. 
          ff$eqt^2 -> 1 - ee$eqt^2 - gg$eqt^2 // FullSimplify], 1], 
     Length[eqssu2]][[1, 1]]]];

soleqssu2$aa1 = 
  Solve[Cases[soleqssu2[[1]], Sqrt[xx$eqt_] -> xx$eqt, -1] == 0, 
    aa1$eqt][[-1]];

soleqssu2ta = 
  If[(aa1$eqt /. soleqssu2$aa1) ∈ Reals, 
   Evaluate[Join[soleqssu2$aa1, soleqssu2 /. soleqssu2$aa1]], 
   Evaluate[Join[{aa1$eqt -> 0}, soleqssu2 /. aa1$eqt -> 0]]];

getsu2frombivec[bivec_, sola_ : True] := 
 Module[{aaa = 
     Thread[{ee$eqt, ff$eqt, gg$eqt} -> 
       Re[Tr[bivec . #] & /@ Table[PauliMatrix[i], {i, 1, 3}] ]]},
   If[(gg$eqt /. aaa) === -1, I*PauliMatrix[2],
    {{aa1$eqt + I aa2$eqt, 
       bb1$eqt + I bb2$eqt}, {-bb1$eqt + I bb2$eqt, 
       aa1$eqt - I aa2$eqt}} /. (If[sola, soleqssu2ta, soleqssu2] /. 
       Thread[{ee$eqt, ff$eqt, gg$eqt} -> 
         Re[
          Tr[bivec . #] & /@ Table[PauliMatrix[i], {i, 1, 3}] ]])]] //
   FullSimplify

In [37]:
eqtsu11 = 
  DeleteCases[
   DeleteDuplicates[
    Thread[(FullSimplify[{Re[#], Im[#]} & /@ 
           ExpandAll[
            Join[
               Flatten[((I {{a$eqt, b$eqt}, {Conjugate[b$eqt], 
                    Conjugate[a$eqt]}} . PauliMatrix[1] . 
                    Inverse[{{a$eqt, b$eqt}, {Conjugate[b$eqt], 
                    Conjugate[a$eqt]}}] - 
                    Total[{ee$eqt, ff$eqt, gg$eqt}*su11sigma] /. 
                   a$eqt Conjugate[a$eqt] - b$eqt Conjugate[b$eqt] -> 
                    1))], {a$eqt Conjugate[a$eqt] - 
                 b$eqt Conjugate[b$eqt] - 1}] /. 
              a$eqt -> aa1$eqt + I aa2$eqt /. 
             b$eqt -> bb1$eqt + I bb2$eqt], {aa1$eqt, aa2$eqt, 
            bb1$eqt, bb2$eqt, ee$eqt, ff$eqt, gg$eqt} \[Element] 
           Reals] // Flatten) == 0] // FullSimplify], True];

eqssu11 = 
  DeleteCases[
   DeleteDuplicates[
    Thread[(FullSimplify[{Re[#], Im[#]} & /@ 
           ExpandAll[
            Join[
               Flatten[(({{a$eqt, b$eqt}, {Conjugate[b$eqt], 
                    Conjugate[a$eqt]}} . su11sigma[[1]] . 
                    Inverse[{{a$eqt, b$eqt}, {Conjugate[b$eqt], 
                    Conjugate[a$eqt]}}] - 
                    Total[{ee$eqt, ff$eqt, gg$eqt}*su11sigma] /. 
                   a$eqt Conjugate[a$eqt] - b$eqt Conjugate[b$eqt] -> 
                    1))], {a$eqt Conjugate[a$eqt] - 
                 b$eqt Conjugate[b$eqt] - 1}] /. 
              a$eqt -> aa1$eqt + I aa2$eqt /. 
             b$eqt -> bb1$eqt + I bb2$eqt], {aa1$eqt, aa2$eqt, 
            bb1$eqt, bb2$eqt, ee$eqt, ff$eqt, gg$eqt} \[Element] 
           Reals] // Flatten) == 0] // FullSimplify], True];

soleqtsu11$eqs = (Solve[
        eqtsu11[[1 ;; 3]], {aa1$eqt, aa2$eqt, bb1$eqt, bb2$eqt}] //. 
       ee$eqt^2 -> ff$eqt^2 + gg$eqt^2 - 1 // FullSimplify) //. 
    ee$eqt^2 -> ff$eqt^2 + gg$eqt^2 - 1 // FullSimplify;

soleqtsu11 = 
  soleqtsu11$eqs[[
   Position[
     Total[
      Transpose@
       Boole[(eqtsu11 /. soleqtsu11$eqs //. 
             ee$eqt^2 -> ff$eqt^2 + gg$eqt^2 - 1 // FullSimplify) //. 
          ee$eqt^2 -> ff$eqt^2 + gg$eqt^2 - 1 // FullSimplify], 1], 
     Length[eqssu11]][[2, 1]]]];

soleqtsu11$aa1 = 
  Solve[Cases[soleqtsu11[[1]], Sqrt[xx$eqt_] -> xx$eqt, -1] == 0, 
    aa1$eqt][[-1]];

soleqtsu11ta = 
  If[(aa1$eqt /. soleqtsu11$aa1) \[Element] Reals, 
   Evaluate[Join[soleqtsu11$aa1, soleqtsu11 /. soleqtsu11$aa1]], 
   Evaluate[Join[{aa1$eqt -> 0}, soleqtsu11 /. aa1$eqt -> 0]]];

soleqssu11$eqs = (Solve[
        eqssu11[[1 ;; 3]], {aa1$eqt, aa2$eqt, bb1$eqt, bb2$eqt}] //. 
       ff$eqt^2 -> ee$eqt^2 - gg$eqt^2 - 1 // FullSimplify) //. 
    ff$eqt^2 -> ee$eqt^2 - gg$eqt^2 - 1 // FullSimplify;

soleqssu11 = 
  soleqssu11$eqs[[
   Position[
     Total[
      Transpose@
       Boole[(eqssu11 /. soleqssu11$eqs //. 
             ff$eqt^2 -> ee$eqt^2 - gg$eqt^2 - 1 // FullSimplify) //. 
          ff$eqt^2 -> ee$eqt^2 - gg$eqt^2 - 1 // FullSimplify], 1], 
     Length[eqssu11]][[1, 1]]]];

soleqssu11$aa1 = 
  Solve[Cases[soleqssu11[[1]], Sqrt[xx$eqt_] -> xx$eqt, -1] == 0, 
    aa1$eqt][[-1]];

soleqssu11ta = 
  If[(aa1$eqt /. soleqssu11$aa1) \[Element] Reals, 
   Evaluate[Join[soleqssu11$aa1, soleqssu11 /. soleqssu11$aa1]], 
   Evaluate[Join[{aa1$eqt -> 0}, soleqssu11 /. aa1$eqt -> 0]]];

getsu11frombivecm[bivec_, sola_ : True] := 
 Block[{soleqs, soleqt, aaa}, 
   If[sola, soleqs = soleqssu11ta; soleqt = soleqtsu11ta;, 
    soleqs = soleqssu11, soleqt = soleqtsu11]; 
   If[Sign[Re[Tr[bivec . bivec]]] >= 0, 
    If[Re[Tr[bivec . su11sigma[[1]]]]*
       Tr[su11sigma[[1]] . su11sigma[[1]]] > 
      0, {{aa1$eqt + I aa2$eqt, 
        bb1$eqt + I bb2$eqt}, {bb1$eqt - I bb2$eqt, 
        aa1$eqt - I aa2$eqt}} /. (soleqs /. 
        Thread[{ee$eqt, ff$eqt, gg$eqt} -> 
          1/2 Re[Tr[# . #] & /@ su11sigma ]*
           Re[Tr[bivec . #] & /@ su11sigma ]]), {{aa1$eqt + I aa2$eqt,
         bb1$eqt + I bb2$eqt}, {bb1$eqt - I bb2$eqt, 
        aa1$eqt - I aa2$eqt}} /. (soleqs /. 
        Thread[{ee$eqt, ff$eqt, 
           gg$eqt} -> -1/2 Re[Tr[# . #] & /@ su11sigma ]*
           Re[Tr[bivec . #] & /@ su11sigma ]])], 
    aaa = 
     Thread[{ee$eqt, ff$eqt, gg$eqt} -> 
       1/2 Re[Tr[# . #] & /@ su11sigma ] Re[
         Tr[bivec . #] & /@ su11sigma ]]; 
    If[(ff$eqt /. aaa) === -1, 
     I*PauliMatrix[
       3], {{aa1$eqt + I aa2$eqt, 
        bb1$eqt + I bb2$eqt}, {bb1$eqt - I bb2$eqt, 
        aa1$eqt - I aa2$eqt}} /. (soleqt /. aaa)]]] // FullSimplify

In [48]:
getsufrombivec[bivec1_, sgndet_, sola_ : True] := If[sgndet > 0, getsu2frombivec[bivec1, sola], getsu11frombivecm[bivec1, sola]]

In [49]:
getxifromsu[hsu_, sgndet_ : 1, facesign_ : 1, tetsign0_ : 1] := 
 If[sgndet > 0, {Transpose[hsu . {{1}, {0}}] // Flatten, 
   Transpose[hsu . {{0}, {1}}] // Flatten}, 
  If[facesign > 0, 
   If[tetsign0 > 0, {Transpose[hsu . {{1}, {0}}] // Flatten, 
     Transpose[hsu . {{0}, {1}}] // 
      Flatten}, {Transpose[hsu . {{0}, {1}}] // Flatten, 
     Transpose[hsu . {{1}, {0}}] // Flatten}], {Transpose[
      1/Sqrt[2] hsu . {{1}, {1}}] // Flatten, 
    Transpose[1/Sqrt[2] hsu . {{1}, {-1}}] // Flatten}]]

In [50]:
bivecdual[bivec_] := HodgeDual[η . bivec] . η

In [51]:
getsolution[bdypoints_] := 
 Block[{edgevec, tetnormalvec, solgsl2c, solgso13, areaef, areaef550, 
   sgndet, fourfacenormal, threedtetfacenormal, threededgevec, bdysu, 
   threeto4dedgevec, tetareasign , ll$111 = Length[bdypoints], re$111,
    aainter$121, areaef55, threeto4dtetfacenormal, bdybivec, 
   bdybivec55, bdyxin, signspp, signpossi, kappaall, fullkappa, 
   tetn0sign, bdybivec55s, rand, matrix},
  
  edgevec = 
   Table[(bdypoints[[#[[1]]]] - bdypoints[[#[[2]]]]) & /@ 
     edgeor[[j]], {j, 1, ll$111}]; 
  tetnormalvec = FullSimplify@gettetoutnormalvec1[edgevec, bdypoints];
   solgsl2c = 
   Table[FullSimplify@getsl2c[tetnormalvec[[i]]], {i, 1, ll$111}]; 
  solgso13 = 
   Table[FullSimplify[getso13@solgsl2c[[i]]], {i, 1, ll$111}];
  
  areaef = 
   Table[
     getarea[
      getoutpfacevec[get3dtet[edgevec[[i]], solgso13[[i]]]]], {i, 1, 
      ll$111}] // FullSimplify;
  areaef55 = Table[Insert[areaef[[i]], 0, i], {i, 1, 5}]; 
  sgndet = 
   Table[
    Det[getoutpfacevec[
       get3dtet[edgevec[[i]], solgso13[[i]]] // Chop][[2]]], {i, 1, 
     ll$111}];
  threedtetfacenormal = 
   Table[
    FullSimplify[
     getfacenormal@
      getoutpfacevec[(get3dtet[edgevec[[i]], solgso13[[i]]])]], {i, 1,
      ll$111}];
  threeto4dtetfacenormal = 
   Table[
    threetofour[#, sgndet[[i]]] & /@ threedtetfacenormal[[i]], {i, 1, 
     ll$111}];
  tetareasign = 
   Table[
    Insert[
     sgndet[[i]]*(# . \[Eta] . # & /@ threeto4dtetfacenormal[[i]]) // 
        N // Chop // Round, 1, i], {i, ll$111}];
  
  If[True,
   threededgevec = 
    Table[(get3dtet[edgevec[[i]], solgso13[[i]]])[[1]], {i, 1, 
      ll$111}]; 
   threeto4dedgevec = 
    Table[
     threetofour[#, sgndet[[i]]] & /@ threededgevec[[i]], {i, 1, 
      ll$111}]; 
   bdybivec = 
    Table[
     getbivec[threeto4dedgevec [[i, #[[1]]]], 
        threeto4dedgevec [[i, #[[2]]]]] & /@ etof, {i, 1, ll$111}]; 
   bdybivec55 = 
    Table[
     Insert[#/Sqrt[Abs[1/2 Tr[# . #]]] & /@ bdybivec[[i]], Jvec[[3]], 
      i], {i, 1, 5}]; 
   bdybivec55s = 
    Table[
     bivec1tohalf[#] & /@ (bdybivec55[[i]] // Normal), {i, 1, ll$111}];
   tetn0sign = 
    Table[
     Sign[#[[1, 1]] + 1/2 // N // Chop] & /@ (bdybivec55s)[[i]], {i, 
      1, ll$111}];
   
   bdysu = 
    Table[
     getsufrombivec[bdybivec55s[[i, j]], sgndet[[i]]], {i, 1, 
      ll$111}, {j, 1, ll$111}];
   bdyxin = 
    Table[
     getxifromsu[bdysu[[i, j]], sgndet[[i]], tetareasign[[i, j]], 
      tetn0sign[[i, j]]], {i, 1, ll$111}, {j, 1, ll$111}];
   
   signspp = 
    Table[
     signpossi = 
      Flatten[
       Table[
        Insert[{i1, i2, i3, i4}, 0, 
         i], {i1, {-1, 1}}, {i2, {-1, 1}}, {i3, {-1, 1}}, {i4, {-1, 
          1}}], 3]; 
     Table[
      If[
       Abs[
         N@
          Total[
           Flatten@
            FullSimplify[
             Total[bdybivec55s[[i]]*signpossi[[j]]*areaef55[[i]]]]]] <
         10^-5, signpossi[[j]], Unevaluated[Sequence[]]], {j, 
       Length[signpossi]}], {i, Length[bdybivec55s]}];
   
   kappaall = 
    Flatten[
     Table[
      matrix = 
       Join[{signspp[[1, i1]]}, {signspp[[2, i2]]}, {signspp[[3, 
          i3]]}, {signspp[[4, i4]]}, {signspp[[5, i5]]}]; 
      If[AntisymmetricMatrixQ[matrix], matrix, 
       Unevaluated[Sequence[]]], {i1, Length[signspp[[1]]]}, {i2, 
       Length[signspp[[2]]]}, {i3, Length[signspp[[3]]]}, {i4, 
       Length[signspp[[4]]]}, {i5, Length[signspp[[5]]]}], 4];
   
   fullkappa = 
    Table[
      If[
       And @@ 
        Table[
         Count[kappaall[[j, i]], 1] == 2, {i, Length[kappaall[[j]]]}],
        kappaall[[j]], Unevaluated@Sequence[]], {j, 
       Length[kappaall]}][[2]];
   
   re$111 = {solgsl2c, bdyxin, bdysu, bdybivec55s, areaef55, 
     fullkappa, threedtetfacenormal, {sgndet, tetn0sign, tetareasign}},
   
   Print["The bdy data is not allowed, there are timelike faces"]; 
   re$111 = 0
   ];
  
  re$111
  ]

In [52]:
getz[gsl2c_, bdxi_] := #/#[[1]] & @ (Inverse[ConjugateTranspose[gsl2c]] . bdxi);

## Get bdry data and Critical Points

In [62]:
Get["/home/dqu/Twocube/FixOrentationKappa/Vertices.wl"];

In [71]:
Vertex1 = Flatten[Table[{i1, i2, i3, i4}, {i1, {0, 1/2}}, {i2, {0, 1}}, {i3, {0, 1}}, {i4, {0, 1}}], 3];

In [72]:
Vertex2 = Flatten[Table[{i1, -i2, i3, i4}, {i1, {0, 1/2}}, {i2, {0, 1}}, {i3, {0, 1}}, {i4, {0, 1}}], 3];

In [134]:
bdypoints = Join[ParallelTable[Vertex1[[#]] & /@ FourSimplices[[i]], {i, (Length@FourSimplices)/2}], 
ParallelTable[Vertex2[[#]] & /@ (FourSimplices[[i]]/.Thread[Table[i, {i, 17, Length@FourSimplices}]->Table[i, {i, 1, 16}]]), {i, 17, Length@FourSimplices}]];

In [135]:
DistanceSquare[v1_, v2_] := (v1 - v2) . DiagonalMatrix[{-1, 1, 1, 1}] . (v1 - v2)

In [136]:
bdydata = ParallelTable[Off[Solve::svars];Off[Solve::ratnz]; getsolution[bdypoints[[i]]], {i, Length@FourSimplices}];

LinkObject::linkv: Argument Null in LinkWrite[Null, MessagePacket[Insert, ins]] is not a valid LinkObject.

LinkObject::linkv: Argument Null in LinkWrite[Null, MessagePacket[Part, partw]] is not a valid LinkObject.

LinkObject::linkv: Argument Null in LinkWrite[Null, MessagePacket[Insert, ins]] is not a valid LinkObject.

General::stop: Further output of LinkObject::linkv will be suppressed during this calculation.

Part::partw:                              1                                                                                1                                                                                  1                                                                                 1                                                                                   1                                                                                1                                                                                  1                                                                                 1                                                                                               Tr[Null . {{1, 0}, {0, 1}} . ConjugateTranspose[Null] . {{-1, 0}, {0, 1}}] - Tr[Null . {{1, 0}, {0, 1}} . ConjugateTranspose[Null] . {{0, I}, {-I, 0}}]  Tr[Null . {{0, -1}, {-1, 0}} . ConjugateTranspose[Null] . {{-1, 0}, {0, 1}}] - Tr[Null . {{0, -1}, {-1, 0}} . ConjugateTranspose[Null

...

General::stop: Further output of Part::partw will be suppressed during this calculation.

Insert::ins:                                                             1                                                                                1                                                                                  1                                                                                 1                                                                                 Tr[Null . {{1, 0}, {0, 1}} . ConjugateTranspose[Null] . {{0, -1}, {-1, 0}}]  Tr[Null . {{0, -1}, {-1, 0}} . ConjugateTranspose[Null] . {{0, -1}, {-1, 0}}]  Tr[Null . {{0, I}, {-I, 0}} . ConjugateTranspose[Null] . {{0, -1}, {-1, 0}}]  Tr[Null . {{-1, 0}, {0, 1}} . ConjugateTranspose[Null] . {{0, -1}, {-1, 0}}]                  Tr[Null . {{1, 0}, {0, 1}} . ConjugateTranspose[Null] . {{-1, 0}, {0, 1}}] + Tr[Null . {{1, 0}, {0, 1}} . ConjugateTranspose[Null] . {{0, -1}, {-1, 0}}]  Tr[Null . {{0, -1}, {-1, 0}} . ConjugateTranspose[Null] . {{-1, 0}, {0, 1}}] + Tr[Null . {{0, -1}, {-1, 0}} . Conjuga

Insert::ins:                                                             1                                                                                1                                                                                  1                                                                                 1                                                                                 Tr[Null . {{1, 0}, {0, 1}} . ConjugateTranspose[Null] . {{0, -1}, {-1, 0}}]  Tr[Null . {{0, -1}, {-1, 0}} . ConjugateTranspose[Null] . {{0, -1}, {-1, 0}}]  Tr[Null . {{0, I}, {-I, 0}} . ConjugateTranspose[Null] . {{0, -1}, {-1, 0}}]  Tr[Null . {{-1, 0}, {0, 1}} . ConjugateTranspose[Null] . {{0, -1}, {-1, 0}}]                  Tr[Null . {{1, 0}, {0, 1}} . ConjugateTranspose[Null] . {{0, -1}, {-1, 0}}] + Tr[Null . {{1, 0}, {0, 1}} . ConjugateTranspose[Null] . {{0, I}, {-I, 0}}]  Tr[Null . {{0, -1}, {-1, 0}} . ConjugateTranspose[Null] . {{0, -1}, {-1, 0}}] + Tr[Null . {{0, -1}, {-1, 0}} . Conjug

Insert::ins: Cannot insert at position {3} in Round[Det[getoutpfacevec[{{-0.5 Tr[Null . {{1., 0}, {0, 1.}} . ConjugateTranspose[Null] . {{-1., 0}, {0, 1.}}], -0.5 Tr[Null . {{0, -1.}, {-1., 0}} . ConjugateTranspose[Null] . {{-1., 0}, {0, 1.}}], -0.5 Tr[Null . {{0, 0. + 1. I}, {0. - 1. I, 0}} . ConjugateTranspose[Null] . {{-1., 0}, {0, 1.}}], -0.5 Tr[Null . {{-1., 0}, {0, 1.}} . ConjugateTranspose[Null] . {{-1., 0}, {0, 1.}}]}, {0.5 Tr[Null . {{1., 0}, {0, 1.}} . ConjugateTranspose[Null] . {{0, -1.}, {-1., 0}}], 0.5 Tr[Null . {{0, -1.}, {-1., 0}} . ConjugateTranspose[Null] . {{0, -1.}, {-1., 0}}], 0.5 Tr[Null . {{0, 0. + 1. I}, {0. - 1. I, 0}} . ConjugateTranspose[Null] . {{0, -1.}, {-1., 0}}], 0.5 Tr[Null . {{-1., 0}, {0, 1.}} . ConjugateTranspose[Null] . {{0, -1.}, {-1., 0}}]}, {0, 0, 0, 0}, {0.5 (Tr[Null . {{1., 0}, {0, 1.}} . ConjugateTranspose[Null] . {{-1., 0}, {0, 1.}}] + Tr[Null . {{1., 0}, {0, 1.}} . ConjugateTranspose[Null] . {{0, -1.}, {-1., 0}}]), 0.5 (Tr[Null . {{0, -1.}, {

General::stop: Further output of Insert::ins will be suppressed during this calculation.

## Find Boundary Faces, Bulk Faces and corresponding positions

In [91]:
Tets = Subsets[#, {4}] & /@ FourSimplices;

In [92]:
TetFaces0 = 
  Table[Subsets[Tets[[i, j]], {3}], {i, Length[FourSimplices]}, {j, 
    Length[Tets[[i]]]}];
TetFaces = 
  Table[Table[
    Insert[TetFaces0[[i, j]], {0, 0, 0}, j], {j, 
     Length@TetFaces0[[i]]}], {i, Length[FourSimplices]}];

In [98]:
kappa0 = Table[bdydata[[i, -3]], {i, Length@bdydata}];

In [99]:
kappa = Block[{savedTet, length, count, kappatest, pos0, pos1}, 
   kappatest = kappa0;
   savedTet = {1};
   count = 0;
   While[Length[savedTet] < Length[FourSimplices], count++; 
    Table[pos0 = Position[Tets, Tets[[savedTet[[count]], j]]];
     If[(Length@pos0) > 1, 
      pos1 = DeleteCases[pos0, {savedTet[[count]], j}][[1]];
      If[MemberQ[savedTet, pos1[[1]]], Unevaluated[Sequence[]], 
       AppendTo[savedTet, pos1[[1]]]; 
       If[
        Delete[TetFaces[[savedTet[[count]], j]], {j}] == 
         Delete[TetFaces[[Sequence @@ pos1]], {pos1[[2]]}], 
        If[
         Delete[kappatest[[savedTet[[count]], j]], {j}] == 
          Delete[kappatest[[Sequence @@ pos1]], {pos1[[2]]}], 
         kappatest[[pos1[[1]]]] = -kappatest[[pos1[[1]]]], 
         Unevaluated[Sequence[]]], Print["faces do not mathch!"]]], 
      Unevaluated[Sequence[]]], {j, 5}]; Print[savedTet]];
   kappatest
   ];

{1, 17, 10}
{1, 17, 10, 26}
{1, 17, 10, 26, 9, 12, 14}
{1, 17, 10, 26, 9, 12, 14, 25, 28, 30}
{1, 17, 10, 26, 9, 12, 14, 25, 28, 30, 11, 13, 4}
{1, 17, 10, 26, 9, 12, 14, 25, 28, 30, 11, 13, 4, 16, 6}
{1, 17, 10, 26, 9, 12, 14, 25, 28, 30, 11, 13, 4, 16, 6, 7}
{1, 17, 10, 26, 9, 12, 14, 25, 28, 30, 11, 13, 4, 16, 6, 7, 27, 29, 20}
{1, 17, 10, 26, 9, 12, 14, 25, 28, 30, 11, 13, 4, 16, 6, 7, 27, 29, 20, 32, 
 
>   22}
{1, 17, 10, 26, 9, 12, 14, 25, 28, 30, 11, 13, 4, 16, 6, 7, 27, 29, 20, 32, 
 
>   22, 23}
{1, 17, 10, 26, 9, 12, 14, 25, 28, 30, 11, 13, 4, 16, 6, 7, 27, 29, 20, 32, 
 
>   22, 23, 2, 15}
{1, 17, 10, 26, 9, 12, 14, 25, 28, 30, 11, 13, 4, 16, 6, 7, 27, 29, 20, 32, 
 
>   22, 23, 2, 15, 3}
{1, 17, 10, 26, 9, 12, 14, 25, 28, 30, 11, 13, 4, 16, 6, 7, 27, 29, 20, 32, 
 
>   22, 23, 2, 15, 3}
{1, 17, 10, 26, 9, 12, 14, 25, 28, 30, 11, 13, 4, 16, 6, 7, 27, 29, 20, 32, 
 
>   22, 23, 2, 15, 3, 5}
{1, 17, 10, 26, 9, 12, 14, 25, 28, 30, 11, 13, 4, 16, 6, 7, 27, 29, 20, 32, 
 
>   22

## all bulk faces, boundary faces and their corresponding position

In [102]:
Triangles = 
  DeleteDuplicates@
   Flatten[
    Table[
     Subsets[Tets[[i, j]], {3}], {i, Length@Tets}, {j, 
      Length@Tets[[i]]}], 2];

In [103]:
FacePosition = Position[TetFaces, #] & /@ Triangles;

In [104]:
BDFaces = 
  Table[pos0 = 
    Tets[[Sequence @@ (#[[1 ;; 2]])]] & /@ FacePosition[[i]]; 
   pp1 = DeleteDuplicates@pos0; 
   If[Length[pp1] > Length[pos0]/2, Triangles[[i]], 
    Unevaluated[Sequence[]]], {i, Length@FacePosition}];

In [105]:
BDFacesPosition = Position[TetFaces, #] & /@ BDFaces;

In [107]:
BulkFaces = 
  Table[pos0 = 
    Tets[[Sequence @@ (#[[1 ;; 2]])]] & /@ FacePosition[[i]]; 
   pp1 = DeleteDuplicates@pos0; 
   If[Length[pp1] == Length[pos0]/2, Triangles[[i]], 
    Unevaluated[Sequence[]]], {i, Length@FacePosition}];

In [108]:
BulkFacesPosition = Position[TetFaces, #] & /@ BulkFaces;

In [110]:
sharedTets = 
  Flatten[
   Table[
    Intersection[Tets[[i]], Tets[[j]]], {i, Length@FourSimplices}, {j,
      i + 1, Length@FourSimplices}], 2];
sharedTetsPos = Position[Tets, #] & /@ sharedTets;

In [113]:
orderBulk[FacesPosition_, k_] := 
 Block[{selectlinks, templist, pos, pos2, test2, templist2, testkappa,
    tempfaces, posnew}, 
  selectlinks = 
   If[MemberQ[sharedTetsPos, #], #, Unevaluated[Sequence[]]] & /@ 
    Subsets[DeleteDuplicates@FacesPosition[[k, All, 1 ;; 2]], {2}]; 
  templist = selectlinks[[1]]; 
  While[(Length@templist) < (Length@Flatten[selectlinks, 1]), 
   pos = Position[selectlinks, templist[[-1, 1]]];
   If[MemberQ[templist, selectlinks[[#[[1]], #[[2]]]]], 
      Unevaluated[Sequence[]], 
      AppendTo[templist, selectlinks[[#[[1]], #[[2]]]]]; 
      AppendTo[templist, 
       selectlinks[[#[[1]], 
        Sequence @@ Complement[{1, 2}, {#[[2]]}]]]]] & /@ pos];
  tempfaces = FacesPosition[[k, All, 1 ;; 2]];
  posnew = Position[tempfaces, #][[1, 1]] & /@ templist;
  FacesPosition[[k, #]] & /@ posnew
  ]

In [114]:
orderBDry[FacesPosition_, k_] := 
 Block[{innerorder, comp}, 
  If[Length@(FacesPosition[[k]]) == 2, FacesPosition[[k]], 
   innerorder = orderBulk[FacesPosition, k];
   comp = Complement[FacesPosition[[k]], innerorder];
   If[comp[[1, 1]] == innerorder[[1, 1]], 
    Join[{comp[[1]]}, innerorder, {comp[[2]]}], 
    Join[{comp[[2]]}, innerorder, {comp[[1]]}]]
   ]
  ]

In [116]:
OrderBulkFaces = Table[orderBulk[BulkFacesPosition, j], {j, Length@BulkFacesPosition}];

In [117]:
OrderBDryFaces = Table[orderBDry[BDFacesPosition, i], {i, Length[BDFacesPosition]}];

In [119]:
Save["/home/dqu/Twocube/FixOrentationKappa/" <> "Labels.wl", {FourSimplices, Vertex1, Vertex2, bdypoints, OrderBDryFaces, OrderBDryFaces, TetFaces, kappa, sharedTets, sharedTetsPos}];